# 1.

In [ ]:
import numpy as np

true_b = 1
true_w = 2

N = 100

np.random.seed(42)
x = np.random.rand(N,1)  # Uniform
epsilon = (0.1 * np.random.randn(N,1))  # standard normal dist  # noise
y = true_b + true_w * x + epsilon  # data genneration

# Shuffles the indices
idx = np.arange(N)
split_index = int(N * 0.8)

train_idx = idx[:split_index]
val_idx = idx[split_index:]

# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

# create tensor ar GPU:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # GPU가 현재 연결이 되어있다면 cuda를 콜하겠다. GPU에 내 데이터를 올릴거예요.
x_train_tensor = torch.as_tensor(x_train).to(device)
y_train_tensor = torch.as_tensor(y_train).to(device)

x_val_tensor = torch.as_tensor(x_val).to(device)
y_val_tensor = torch.as_tensor(y_val).to(device)

# train data 로 구한 parameter
@timer
def train_model_torch(lr = 0.1, epochs=1000):
  # Initialize parameters
  b_hat = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
  w_hat = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)

  for epoch in range(epochs):
    # Loss computation
    y_hat = b_hat + w_hat * x_train_tensor
    error = (y_hat - y_train_tensor)
    mse_loss = torch.mean(error ** 2)
    # Gradient computation and descent
    mse_loss.backward()
    with torch.no_grad(): # 경사 하강법 할 땐 auto grad 끄고 하는 게 좋음. 아니면 뻑남
      b_hat -= lr * b_hat.grad  # in-place operation 써줘야함.
      w_hat -= lr * w_hat.grad
    b_hat.grad.zero_() # gradient 초기화 시켜줘야함. 안 하면 누적합으로 계산하기 때문에
    w_hat.grad.zero_()

  mse_loss = nn.MSELoss()
  y_hat = b_hat + w_hat * x_val_tensor
  loss = mse_loss(y_hat, y_val_tensor)
  return b_hat, w_hat, loss

b_hat, w_hat, loss = train_model_torch()
print(loss)

# 2.

In [ ]:
import pickle
# 데이터 로드
with open('quiz_data.pkl', 'rb') as f:
	data = pickle.load(f)

x = data["x"]
y = data["y"]

# Shuffles the indices
idx = np.arange(100)
split_index = int(100 * 0.8)

train_idx = idx[:split_index]
val_idx = idx[split_index:]

# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

# create tensor ar GPU:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # GPU가 현재 연결이 되어있다면 cuda를 콜하겠다. GPU에 내 데이터를 올릴거예요.
x_train_tensor = torch.as_tensor(x_train).to(device)
y_train_tensor = torch.as_tensor(y_train).to(device)

x_val_tensor = torch.as_tensor(x_val).to(device)
y_val_tensor = torch.as_tensor(y_val).to(device)

sum = 0
for i in range(10):

  def train_model_torch(lr = 0.1, epochs=1000):
    # Initialize parameters
    b_hat = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
    w_hat = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)

    for epoch in range(epochs):
      # Loss computation
      y_hat = b_hat + w_hat * x_train_tensor
      error = (y_hat - y_train_tensor)
      mse_loss = torch.mean(error ** 2)
      # Gradient computation and descent
      mse_loss.backward()
      with torch.no_grad(): # 경사 하강법 할 땐 auto grad 끄고 하는 게 좋음. 아니면 뻑남
        b_hat -= lr * b_hat.grad  # in-place operation 써줘야함.
        w_hat -= lr * w_hat.grad
      b_hat.grad.zero_() # gradient 초기화 시켜줘야함. 안 하면 누적합으로 계산하기 때문에
      w_hat.grad.zero_()

    mse_loss = nn.MSELoss()
    y_hat = b_hat + w_hat * x_val_tensor
    loss = mse_loss(y_hat, y_val_tensor)
    return b_hat, w_hat, loss

  b_hat, w_hat, loss = train_model_torch()
  sum += loss

print(sum / 10)